<a href="https://colab.research.google.com/github/kamalkraj/minGPT-TF/blob/master/play_char.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install fastprogress==0.2.3

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [2]:
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [3]:
import math
import numpy as np
import tensorflow as tf
from mingpt.model import GPT, GPTConfig

In [4]:
class CharDataset:

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return math.ceil(len(self.data) / (self.block_size + 1))

    def __iter__(self):
        # we're actually going to "cheat" and pick a spot in the dataset at random
        for _ in range(self.__len__()):
            i = np.random.randint(0, len(self.data) - (self.block_size + 1))
            chunk = self.data[i:i+self.block_size+1]
            dix = [self.stoi[s] for s in chunk]
            x = tf.convert_to_tensor(dix[:-1], dtype=tf.int32)
            y = tf.convert_to_tensor(dix[1:], dtype=tf.int32)
            yield x, y
    
    __call__ = __iter__

In [5]:
block_size = 128 

In [6]:
text = open('input.txt', 'r').read()
train_dataset_gen = CharDataset(text, block_size) 

data has 1115395 characters, 65 unique.


In [7]:
train_dataset = tf.data.Dataset.from_generator(train_dataset_gen,(tf.int32,tf.int32))

In [8]:
from mingpt.model import GPT, GPTConfig
mconf = GPTConfig(train_dataset_gen.vocab_size, train_dataset_gen.block_size,
                  n_layer=8, n_head=8, n_embd=512)

In [9]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=10, batch_size=64, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=200*len(train_dataset_gen)*block_size,
                      num_workers=4)
trainer = Trainer(GPT, mconf, train_dataset, len(train_dataset_gen), None, None, tconf)

In [10]:
trainer.train()

epoch 1: train loss 350.23410. lr 5.999636e-04
epoch 2: train loss 295.87991. lr 5.998533e-04
epoch 3: train loss 260.77359. lr 5.996690e-04
epoch 4: train loss 235.39684. lr 5.994107e-04
epoch 5: train loss 217.30246. lr 5.990785e-04
epoch 6: train loss 204.98639. lr 5.986725e-04
epoch 7: train loss 195.28625. lr 5.981929e-04
epoch 8: train loss 188.95831. lr 5.976396e-04
epoch 9: train loss 183.35428. lr 5.970130e-04
epoch 10: train loss 178.87193. lr 5.963130e-04


In [11]:
# alright, let's sample some character-level shakespear
from mingpt.utils import sample

context = "O God, O God!"
x = tf.convert_to_tensor([train_dataset_gen.stoi[s] for s in context], dtype=tf.int32)[None,...]
y = sample(trainer.model, x, 2000, temperature=0.9, sample=True, top_k=5)[0]
completion = ''.join([train_dataset_gen.itos[int(i)] for i in y])
print(completion)

O God, O God! and thou all the place.

CLIFFFORD:
I would be be this stone, but I steel think,
That would be secret his hath hold makes to be proclaimes
And strain thy lips and the ward trouble stay
And not them, I see are aboard.

PERDITA:
The san true too, and less that be the dukest and life, then
that we meast there's be the want of him and heaven
At all this subjects such out a hall about.
Here shall be so true, and we was a word
the prayer of head of my cousin, we make me there lies.

PETRUCHIO:
Then, what now he hope out of thy body's daughter him.

FLUMIA:
This is murderer, though never she him be poilous too:
Then he's a man bed a pleased.

DUKE VINCENTIO:
I pray you, sir, I talke you to the city:
To say, I will say him to the mirdlain arms.

PETRUCHIO:
Why, my lord, that's yet all the seat.

CURTIS:
You're sent? I have been thee between still.

LUCIO:
The words then when thyself, and yet so the work this land.
The life, then, therefore has been have
stone the stear of mine, w